# centiBils: A unit for Investment Performance Management

## Abstract

Percentage Returns are the de-facto standard for Investment Performance
Measurement. Yet they are not very well suited for multi-period performance
evaluation. As an illustrative example, does the sequence (-30%, +40%) represent a
positive or negative overall return? We review some of the problems with using
percentage returns such as the lack of symmetry, lack of additivity and
differences between the arithmetic and geometric means. We then introduce a new
logarithmic unit modeled on the deciBel, tentatively called a centiBil, which
addresses these shortcomings. Finally we conclude by using the centiBil to
compare the performance of US Stocks, Bonds and Bills since 1927 and show how
their use eases cross-period comparisons.

## Agenda

 1. Problems with (Simple-)Returns
      * Returns are not symmetric
      * Returns are not (longitudinally) additive
      * The mean return doesn't represent the growth rate of the investment
 2. Problems with existing logarithmic units
      * Neper
          * lacks interpretability
      * Bels (i.e. deciBels)
          * too large for use with investment returns
 3. centiBils
      * Definition
      * Standard Values
 4. Applications
      * S&P 500 Returns since 1900
      * Comparison of ALSI, ALBI and STEFI returns

## Problems with Returns

### Returns are not symmetric

  * You are probably aware that if your investment experiences a **-50%** return, you need a **+100%** just to break even again.
  * This lack of symmetry makes it difficult to interpret sequences of returns. For example, does the sequence (-30%, +40%) represent an overall positive or negative return?

### Returns are not (longitudinally) additive

  * The overall return of the sequence (-30%, +40%) is calculated as follows:
  
    $$ r = (1 - \frac{30}{100})(1 + \frac{40}{100}) - 1 = -0.02 = -2\% $$

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
%matplotlib inline

In [ ]:
r = (1-30/100)*(1+40/100)-1
print("r = {r:.2f} = {r:.2%}".format(r=r))

### The mean return doesn't represent the growth rate

$$ g = \mu - \frac{1}{2}\sigma^2 $$

#### US Asset Class Returns (since 1927)

In [ ]:
us_data = pd.read_excel('data/histretSP.xls', sheetname='Returns by year', skiprows=17, skip_footer=10, index_col=0)
pd.concat([us_data.head(), us_data.tail()])

In [ ]:
from IPython.core.display import display, HTML
def to_html_perc(df):
    return HTML(df.to_html(formatters={c:"{:,.2%}".format for c in df.columns}))

def calc_metrics(returns):
    df = pd.DataFrame(returns.mean(skipna=False).to_frame().T)
    df.index = ['Mean']
    df.loc['Std Dev.'] = returns.std()
    df.loc['CAGR'] = np.expm1(np.log1p(returns).mean(skipna=False))
    return df

returns = us_data.iloc[:, [0, 2, 1]].copy()
to_html_perc(calc_metrics(returns))

In [ ]:
sp500 = returns['S&P 500']
tbill = returns['3-month T.Bill']
returns['S&P 2X'] = 1*(sp500-sp500.mean())+sp500
to_html_perc(calc_metrics(returns))

In [ ]:
us = us_data[['Stocks', 'T.Bonds', 'T.Bills']].copy()
us.loc[1927] = 100
us.sort_index(inplace=True)
us = us/us.iloc[0]
#us['S&P 2X'] = np.exp(np.cumsum(np.log1p(returns['S&P 2X'])))
pd.concat([us.head(3), us.tail(3)])

In [ ]:
us.plot(figsize=(16,8))

In [ ]:
us.plot(figsize=(16,8), logy=True)

In [ ]:
np.log(us).plot(figsize=(16,8))

## centiBils

### Definition

$$ 1 Bil = \ln_2 2 = \ln_2 (1+1) = \ln (1+100\%) / \ln 2$$

$$ r \implies 100 \ln_2(1+r) \textrm{ centiBils} = \frac{100}{\ln 2} \ln(1+r) \textrm{ cB} $$

In [ ]:
def ret2cb(ret):
    return 100/np.log(2)*np.log1p(ret)

def cb2ret(cb):
    return np.expm1(cb/100*np.log(2))

def idx2cb(idx):
    return 100/np.log(2)*np.log(idx)

def cb2idx(cb):
    return np.exp((c/100*np.log(2)))

## Standard Percentage Returns

In [ ]:
vals = np.array([-1, -0.50, -0.25, -0.20, -0.10, -0.01, -0.0001])

In [ ]:
std_rets = np.r_[vals, 0, -vals[::-1]]
standard_returns = pd.DataFrame({'Percentage Return':std_rets*100, 'centiBils Return':ret2cb(std_rets)})
standard_returns

In [ ]:
std_cbs = 100*np.r_[vals, 0, -vals[::-1]]
standard_centibils = pd.DataFrame({'centiBils Return':std_cbs, 'Percentage Return':cb2ret(std_cbs)*100})
standard_centibils

## Applications

In [ ]:
np.log(us).head()

In [ ]:
idx2cb(us).plot(figsize=(16,8))

In [ ]:
import nvd3
#nvd3.ipynb.initialize_javascript(use_remote=True)
nvd3.ipynb.initialize_javascript()

In [ ]:
from nvd3 import lineChart
chart = lineChart(name='NVD3', x_is_date=False, width=1024)
for asset_class, series in idx2cb(us).iteritems():
    #x = ["{}-12-31".format(dt) for dt in series.index]
    x = [str(dt) for dt in series.index]
    chart.add_serie(name=asset_class, y=list(series.values), x=x)

chart.buildcontent()

In [ ]:
HTML(chart.htmlcontent)